### ナイーブベイズ分類

### ベイズの定理

$P(\boldsymbol{C} \mid \boldsymbol{x}) = \frac{P(\boldsymbol{x} \mid \boldsymbol{C}) \cdot P(\boldsymbol{C})}{P(\boldsymbol{x})} $  

今回求めたいのは、特徴ベクトルが得られたときにどちらに入るかの事後確率(=$P(C_n \mid \boldsymbol{x})$)  
そのため、分母(周辺確率)はどのクラスの確率でも同じなため、比例式に変形可能  

$P(\boldsymbol{C} \mid \boldsymbol{x}) \propto P(\boldsymbol{x} \mid \boldsymbol{C}) \cdot P(\boldsymbol{C})$

ナイーブ(単純)な仮定として、すべての特徴量が互いに独立であるとする：
$P(\boldsymbol{x} \mid \boldsymbol{C}) = \prod_{i=1}^{n} P(x_i \mid \boldsymbol{C})$

事後確率を変形  
$P(\boldsymbol{C} \mid \boldsymbol{x}) \propto P(\boldsymbol{C}) \cdot \prod_{i=1}^{n} P(x_i \mid \boldsymbol{C})$  

それぞれの確率は、小さい値なため、logをとり小さい確率も考慮可能とする。  
$\log P(\boldsymbol{C} \mid \boldsymbol{x}) \propto \log P(\boldsymbol{C}) + \sum_{i=1}^{n} \log P(x_i \mid \boldsymbol{C})$

最後に、ある特徴ベクトルが与えられたときにそれがスパムである確率とスパムでない確率を求め、大きい方のクラスに分類する。  
$\hat{C} = \arg\max_{C} ( \log P(\boldsymbol{C}) + \sum_{i=1}^{n} \log P(x_i \mid \boldsymbol{C}))$


---

$\boldsymbol{x} = (x_1, x_2, \dots, x_n)$ : 説明変数(特徴ベクトル)  
$\boldsymbol{C}$ : 目的変数(スパムorスパムでない)  
$P(\boldsymbol{C} \mid \boldsymbol{x})$：事後確率  
$P(\boldsymbol{C})$：事前確率  
$P(\boldsymbol{x} \mid \boldsymbol{C})$：尤度  
$P(\boldsymbol{x})$：周辺確率  


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# スパムのデータを取得
data = pd.read_csv('spambase_data.txt', header=None)

# カラムがないため、カラム名を設定(spambase.namesを参照)
columns = [
    'word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our', 'word_freq_over', 
    'word_freq_remove', 'word_freq_internet', 'word_freq_order', 'word_freq_mail', 'word_freq_receive', 
    'word_freq_will', 'word_freq_people', 'word_freq_report', 'word_freq_addresses', 'word_freq_free', 
    'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit', 'word_freq_your', 
    'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george', 
    'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet', 'word_freq_857', 'word_freq_data', 
    'word_freq_415', 'word_freq_85', 'word_freq_technology', 'word_freq_1999', 'word_freq_parts', 
    'word_freq_pm', 'word_freq_direct', 'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 
    'word_freq_project', 'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference', 
    'char_freq_;', 'char_freq_(', 'char_freq_[', 'char_freq_!', 'char_freq_$', 'char_freq_#', 
    'capital_run_length_average', 'capital_run_length_longest', 'capital_run_length_total', 'spam'
]
data.columns = columns

In [3]:
# 説明変数
X = pd.DataFrame(data.iloc[:, :-1])
# 目的変数(スパムかどうか)
y = pd.Series(data.iloc[:, -1])
# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# 説明変数をバイナリ化
X_train, X_test = (X_train > 0.0).astype(int), (X_test > 0.0).astype(int)

In [5]:
def fit(x, y):
    prior = {}
    likelihood = {}
    # 事前確率と尤度を計算
    for c in np.unique(y):
        X_c = x[y == c]
        # 事前確率の計算: P(C_m)
        prior[c] = X_c.shape[0] / x.shape[0]
        # 各特徴量の尤度関数: P(X_m|C_n)
        # +1, +2は ラプラス平滑化のための措置(未知のデータでゼロ確率にならないように)
        likelihood[c] = (X_c.sum(axis=0) + 1) / (len(X_c) + 2)

    return prior, likelihood

def predict(x, prior, likelihood):
    pred = []
    # あるサンプルデータについての予測を行う
    for x0 in x:
        # スパムと非スパムの事後確率: P(C_n|X_0, X_1, ..., X_m)
        a = [0, 0]
        for i in np.unique(y):
            for j in range(len(x0)):
                # 対数を取ることで計算の安定性を向上
                a[i] += np.log(likelihood[i][j]) * x0[j]
            # 事前確率を加える
            a[i] += np.log(prior[i])
        # 事後確率が最大のクラスに分類
        pred.append(np.argmax(a))
    return np.array(pred)

In [6]:
# モデルの学習
prior, likelihood = fit(X_train.values, y_train.values)
# モデルの予測
pred = predict(X_test.values, prior, likelihood)
# モデルの評価
print("Accuracy:", accuracy_score(y_test, pred))
# 混同行列と分類レポートの表示
print("Confusion Matrix:")
print(confusion_matrix(y_test, pred))
print("Classification Report:")
print(classification_report(y_test, pred, target_names=['Not Spam', 'Spam']))

Accuracy: 0.8154180238870793
Confusion Matrix:
[[369 162]
 [  8 382]]
Classification Report:
              precision    recall  f1-score   support

    Not Spam       0.98      0.69      0.81       531
        Spam       0.70      0.98      0.82       390

    accuracy                           0.82       921
   macro avg       0.84      0.84      0.82       921
weighted avg       0.86      0.82      0.81       921

